# Importation des bibliothèques

In [1]:
import pandas as pd
import sqlite3
import os
import warnings
warnings.filterwarnings("ignore")

# Connexion à la base de données brutes

In [ ]:
connect = sqlite3.connect("../Databases/raw-database.db")
cursor = connect.cursor()

# Données de matching des matchs Skill Corner / Stats Bomb

In [3]:
matching_matches = pd.read_json("Projet_centres_data/matching_matches.json")

In [4]:
matching_matches.to_sql("matching_matches", con = connect, if_exists = "replace", index = False)

306

# Données de matching des joueurs Skill Corner / Stats Bomb

In [5]:
matching_players = pd.read_json("Projet_centres_data/matching_players.json")

In [6]:
matching_players.to_sql("matching_players", con = connect, if_exists = "replace", index = False)

639

# Données de matching des équipes Skill Corner / Stats Bomb

In [7]:
matching_teams = pd.read_json("Projet_centres_data/matching_teams.json")

In [8]:
matching_teams.to_sql("matching_teams", con = connect, if_exists = "replace", index = False)

18

# Informations sur les matchs Stats Bomb

In [9]:
SB_matches = pd.read_json("Projet_centres_data/SB_matches.json")

In [10]:
SB_matches.to_sql("SB_matches", con = connect, if_exists = "replace", index = False)

306

# Données events de Stats Bomb

In [11]:
liste_fichier_events = os.listdir("Projet_centres_data/SB_events")

In [13]:
events = pd.concat([pd.read_json("Projet_centres_data/SB_events/" + fichier_event) for fichier_event in liste_fichier_events])

In [14]:
colonnes = ["id", "shot_type", "shot_outcome", "type", "match_id", "period", "possession", "location", "pass_cross", "pass_type", "index",
            "pass_end_location", "minute", "shot_end_location", "pass_body_part", "player_id"]

dico_rename = {"index" : "index_event", "id" : "event_id", "match_id" : "match_id_SB"}
events = events[colonnes].reset_index(drop = True).rename(dico_rename, axis = 1)

events_loc = events.pop("location").dropna()
events_loc = pd.DataFrame(events_loc.tolist(), index = events_loc.index, columns = ["x_loc", "y_loc", "z_loc"])

events_pass_loc = events.pop("pass_end_location").dropna()
events_pass_loc = pd.DataFrame(events_pass_loc.tolist(), index = events_pass_loc.index, columns = ["x_pass", "y_pass"])

events_shot_loc = events.pop("shot_end_location").dropna()
events_shot_loc = pd.DataFrame(events_shot_loc.tolist(), index = events_shot_loc.index, columns = ["x_shot", "y_shot", "z_shot"])

events = pd.concat([events, events_loc, events_pass_loc, events_shot_loc], axis = 1)

In [15]:
events.to_sql("events", con = connect, if_exists = "replace", index = False)

1118352

# Données freeze frame de Skill Corner sur les centres

In [16]:
liste_fichier_freeze_frames = os.listdir("Projet_centres_data/SKC_crosses_freeze_frames")

In [17]:
freeze_frames = pd.DataFrame()

for fichier_freeze_frames in liste_fichier_freeze_frames :
    freeze_frames_import = pd.read_json("Projet_centres_data/SKC_crosses_freeze_frames/" + fichier_freeze_frames)
    
    freeze_frames_import["match_id_SKC"] = int(fichier_freeze_frames.replace(".json", ""))

    freeze_frames = pd.concat([freeze_frames, freeze_frames_import])

freeze_frames.reset_index(drop = True, inplace = True)
freeze_frames.drop("image_corners_projection", axis = 1, inplace = True)

In [18]:
def transform_freeze_frames_data(row) :
    df = pd.DataFrame(row.data)
    df["frame"] = row.frame
    df["match_id_SKC"] = row.match_id_SKC
    return df

In [19]:
freeze_frames_possession = freeze_frames.pop("possession")
freeze_frames[["group", "tackable_object"]] = pd.json_normalize(freeze_frames_possession)

freeze_frames_data = pd.concat((freeze_frames.apply(transform_freeze_frames_data, axis = 1)).tolist())
freeze_frames.drop("data", axis = 1, inplace = True)

In [20]:
freeze_frames.to_sql("freeze_frames", con = connect, if_exists = "replace", index = False)

20576

In [21]:
freeze_frames_data.to_sql("freeze_frames_data", con = connect, if_exists = "replace", index = False)

467914

# Fermeture de la base de données

In [22]:
connect.close()